# F-MNIST Multilayer Perceptron
This notebook uses [Flux](https://github.com/FluxML/Flux.jl) to train a multilayer perceptron on the [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset.  It exports the trained model with [FluxJS](https://github.com/FluxML/FluxJS.jl) to [deeplearn.js](https://deeplearnjs.org/) so that it can be used in the browser.  

(Credit for this notebook goes to [WooKyoung Noh](https://github.com/wookay)).

In [2]:
using Flux
using Flux: onehotbatch, argmax, crossentropy, throttle, @epochs
using BSON
using Base.Iterators: repeated
using FluxJS
using MLDatasets # FashionMNIST
using ColorTypes: N0f8, Gray

In [4]:
const Img = Matrix{Gray{N0f8}}

function prepare_train()
    # load full training set
    train_x, train_y = FashionMNIST.traindata() # 60_000

    trainrange = 1:6_000 # 1:60_000
    imgs = Img.([train_x[:,:,i] for i in trainrange])
    # Stack images into one large batch
    X = hcat(float.(reshape.(imgs, :))...) # |> gpu
    # One-hot-encode the labels
    Y = onehotbatch(train_y[trainrange], 0:9) # |> gpu
    X, Y
end

function prepare_test()
    # Load full test set
    test_x,  test_y  = FashionMNIST.testdata() # 10_000

    testrange = 1:1_000 # 1:10_000
    test_imgs = Img.([test_x[:,:,i] for i in testrange])
    tX = hcat(float.(reshape.(test_imgs, :))...) # |> gpu
    tY = onehotbatch(test_y[testrange], 0:9) #|> gpu
    
    # Save the first 100 images in a bson for use in the web demo
    bson("test_images.bson", Dict(
        :images => reshape(Float32.(tX[:,1:100]), 784*100),
        :labels =>Int32.(test_y[1:100])
    ))
    
    tX, tY
end

X, Y = prepare_train()
tX, tY = prepare_test()

m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax) #|> gpu

Chain(Dense(784, 32, NNlib.relu), Dense(32, 10), NNlib.softmax)

In [5]:
loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(argmax(m(x)) .== argmax(y))

dataset = repeated((X, Y), 200)
evalcb = () -> @show(loss(X, Y))
opt = ADAM(params(m))

@epochs 5 Flux.train!(loss, dataset, opt, cb = throttle(evalcb, 2))

println("Training set accuracy: ", accuracy(X, Y))
# 0.983

println("Test set accuracy: ", accuracy(tX, tY))
# 0.83

INFO: Epoch 1


loss(X, Y) = 2.2092898282431657 (tracked)
loss(X, Y) = 0.7711078301661559 (tracked)
loss(X, Y) = 0.5753037785678004 (tracked)
loss(X, Y) = 0.4886423081057284 (tracked)
loss(X, Y) = 0.43305247741176495 (tracked)
loss(X, Y) = 0.39058106078322763 (tracked)


INFO: Epoch 2


loss(X, Y) = 0.3782079092994961 (tracked)
loss(X, Y) = 0.3482974056880615 (tracked)
loss(X, Y) = 0.3216500239810373 (tracked)
loss(X, Y) = 0.2972648955063786 (tracked)
loss(X, Y) = 0.2746521381826436 (tracked)
loss(X, Y) = 0.25406525767792704 (tracked)


INFO: Epoch 3


loss(X, Y) = 0.24590414591071832 (tracked)
loss(X, Y) = 0.2292228276802084 (tracked)
loss(X, Y) = 0.2139464663078373 (tracked)
loss(X, Y) = 0.20096482093341603 (tracked)
loss(X, Y) = 0.18836702544141706 (tracked)
loss(X, Y) = 0.1757916763713962 (tracked)
loss(X, Y) = 0.16621841612299457 (tracked)


INFO: Epoch 4


loss(X, Y) = 0.15601178095725948 (tracked)
loss(X, Y) = 0.14648761848005154 (tracked)
loss(X, Y) = 0.13735999868574397 (tracked)
loss(X, Y) = 0.1288889449222339 (tracked)
loss(X, Y) = 0.12104516419992967 (tracked)


INFO: Epoch 5


loss(X, Y) = 0.11598446578921456 (tracked)
loss(X, Y) = 0.10890311641299735 (tracked)
loss(X, Y) = 0.10323907165672414 (tracked)
loss(X, Y) = 0.09795740634899289 (tracked)
loss(X, Y) = 0.09286789322018227 (tracked)
loss(X, Y) = 0.08781265548413232 (tracked)
Training set accuracy: 0.9836666666666667
Test set accuracy: 0.837


In [6]:
# See the deeplearn.js representation of the model
@code_js m(X[:,1])

let model = (function () {
  let math = dl.ENV.math;
  function barracuda(gnat) {
    return math.add(math.matrixTimesVector(model.weights[0], gnat), model.weights[1]);
  };
  function barracuda(grouse) {
    return math.relu(math.add(math.matrixTimesVector(model.weights[2], grouse), model.weights[3]));
  };
  function model(nightingale) {
    return math.softmax(barracuda(barracuda(nightingale)));
  };
  model.weights = [];
  return model;
})();
flux.fetchWeights("model.bson").then((function (ws) {
  return model.weights = ws;
}));


In [7]:
# Write the model javascript and the model weights to files
FluxJS.compile("mlp", m, X[:,1])